In [1]:
from google.colab import drive

# Mount the Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import torch
from transformers import XLNetTokenizer, XLNetForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import Dataset
import os
import itertools
import json

# Load and preprocess the dataset
file_path = '/content/drive/Shared drives/DATA298B/Readmission/final_enhanced_patient_summaries.json'

# Load JSON data and convert to DataFrame
with open(file_path, 'r') as f:
    data = pd.DataFrame([json.loads(line) for line in f])

# Extract text data and labels
data['text'] = data['enhanced_summary']
data['label'] = data['readmission_status'].apply(lambda x: 1 if x == 'yes' else 0)

# Initialize the tokenizer
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')

# Define a custom dataset class with contiguous tensors
class ReadmissionDataset(Dataset):
    def __init__(self, encodings, labels):
        # Ensure all tensors in encodings and labels are contiguous
        self.encodings = {key: torch.tensor(val).contiguous() for key, val in encodings.items()}
        self.labels = torch.tensor(labels).contiguous()

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

# Define hyperparameters to tune
hyperparameters = {
    'num_train_epochs': [4],
    'per_device_train_batch_size': [16],
    'learning_rate': [3e-5],
}

# Create all combinations of hyperparameters
param_combinations = list(itertools.product(*hyperparameters.values()))

# Stratified cross-validation setup
k_folds = 5
skf = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=42)
metrics_per_combination = {}

# Folder to save the model
model_save_path = '/content/drive/Shared drives/DATA298B/Readmission/Dataset/saved_model9'
os.makedirs(model_save_path, exist_ok=True)

# Perform cross-validation with hyperparameter tuning
for param_idx, params in enumerate(param_combinations):
    print(f"\nTraining with hyperparameters: {params}")

    num_train_epochs, per_device_train_batch_size, learning_rate = params
    fold_metrics = []

    for fold, (train_index, test_index) in enumerate(skf.split(data, data['label'])):
        print(f"\nFold {fold + 1}/{k_folds}")

        # Split the data
        train_texts, test_texts = data['text'].iloc[train_index].tolist(), data['text'].iloc[test_index].tolist()
        train_labels, test_labels = data['label'].iloc[train_index].tolist(), data['label'].iloc[test_index].tolist()

        # Tokenize
        train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
        test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=512)

        # Create dataset
        train_dataset = ReadmissionDataset(train_encodings, train_labels)
        test_dataset = ReadmissionDataset(test_encodings, test_labels)

        # Load the model
        model = XLNetForSequenceClassification.from_pretrained('xlnet-base-cased', num_labels=2)

        # Make all model parameters contiguous
        for param in model.parameters():
            param.data = param.data.contiguous()

        # Set up training arguments
        training_args = TrainingArguments(
            output_dir=f'./results_fold_{fold}',
            num_train_epochs=num_train_epochs,
            per_device_train_batch_size=per_device_train_batch_size,
            per_device_eval_batch_size=16,
            learning_rate=learning_rate,
            evaluation_strategy="no",
            report_to="none",
        )

        # Define the Trainer
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=test_dataset,
        )

        # Train the model
        trainer.train()

        # Save the model after training in each fold
        model.save_pretrained(os.path.join(model_save_path, f'model_fold_{fold + 1}_params_{param_idx}'))
        tokenizer.save_pretrained(os.path.join(model_save_path, f'model_fold_{fold + 1}_params_{param_idx}'))

        # Evaluate the model
        predictions = trainer.predict(test_dataset)
        pred_labels = predictions.predictions.argmax(-1)

        # Calculate metrics
        report = classification_report(test_labels, pred_labels, output_dict=True)
        fold_metrics.append(report)

    # Aggregate metrics for the hyperparameter configuration
    avg_metrics = {
        "precision": sum(d["weighted avg"]["precision"] for d in fold_metrics) / k_folds,
        "recall": sum(d["weighted avg"]["recall"] for d in fold_metrics) / k_folds,
        "f1-score": sum(d["weighted avg"]["f1-score"] for d in fold_metrics) / k_folds,
        "accuracy": sum(d["accuracy"] for d in fold_metrics) / k_folds,
    }

    metrics_per_combination[param_idx] = avg_metrics
    print(f"\nAverage Metrics for combination {param_idx}:")
    print(f"Precision: {avg_metrics['precision']:.4f}")
    print(f"Recall: {avg_metrics['recall']:.4f}")
    print(f"F1-score: {avg_metrics['f1-score']:.4f}")
    print(f"Accuracy: {avg_metrics['accuracy']:.4f}")

# Find the best hyperparameter combination
best_combination_idx = max(metrics_per_combination, key=lambda k: metrics_per_combination[k]['f1-score'])
best_metrics = metrics_per_combination[best_combination_idx]
print(f"\nBest hyperparameter combination: {param_combinations[best_combination_idx]}")
print(f"Best F1-score: {best_metrics['f1-score']:.4f}")


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(



Training with hyperparameters: (4, 16, 3e-05)

Fold 1/5


pytorch_model.bin:   0%|          | 0.00/467M [00:00<?, ?B/s]

Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'logits_proj.weight', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss


...

Step,Training Loss



Fold 5/5


Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'logits_proj.weight', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss



Average Metrics for combination 0:
Precision: 0.9658
Recall: 0.9649
F1-score: 0.9648
Accuracy: 0.9649

Best hyperparameter combination: (4, 16, 3e-05)
Best F1-score: 0.9648


In [ ]:
import torch
import numpy as np
from transformers import XLNetTokenizer, XLNetForSequenceClassification

# Specify the number of folds
num_folds = 5  # Adjust this based on the number of folds used during training
model_paths = [f'/content/drive/Shared drives/DATA298B/Readmission/Dataset/saved_model9/model_fold_{i+1}_params_0' for i in range(num_folds)]

# Load the tokenizer
tokenizer = XLNetTokenizer.from_pretrained(model_paths[0])  # Use the tokenizer from any model

# Example clinical notes for prediction
clinical_notes = [

                  "The patient is a 65-year-old female with a history of congestive heart failure and chronic obstructive pulmonary disease (COPD). She was admitted for acute exacerbation of COPD, presenting with severe shortness of breath and low oxygen saturation. During her hospital stay, her condition required frequent adjustments to her oxygen therapy and multiple high-dose steroid treatments. The discharge plan included oxygen therapy at home and follow-up in one week. However, she lives alone and expressed concerns about managing her medications and oxygen setup. Her last lab results showed borderline low sodium and elevated blood urea nitrogen, with fluctuating blood pressure readings. She also had a recent hospitalization three weeks ago for a similar issue.",
                  "The patient is a 45-year-old male with hypertension and Type 2 diabetes, who was admitted for routine monitoring after an elective orthopedic surgery. His hospital course was stable, with no complications, and he showed consistent improvement in mobility and pain control. Vital signs and lab results were within normal ranges at discharge. He was discharged with a home exercise plan and clear instructions on managing post-operative care, including regular check-ups with his primary physician. He has a strong support system at home, with family members who can assist with his care as needed. No significant health issues were noted at discharge.",
                  "The patient is a 50-year-old male with schizophrenia and Type 2 diabetes, admitted for hyperglycemia and dehydration. He was stabilized with insulin and rehydration during his stay. However, he has a history of non-adherence to medication, which complicates his diabetes management. He is being discharged with insulin therapy and has been referred for mental health follow-up, but he has a limited support system. His lab results at discharge were within normal ranges, but his history of inconsistent medication adherence and frequent admissions suggests a high risk of readmission.",
                  "The patient is a 55-year-old female with no significant prior medical history, admitted for knee replacement surgery. Her postoperative recovery was smooth, with no complications, and her vital signs remained stable throughout her stay. Physical therapy sessions showed good progress, and she was discharged with clear instructions for exercises and a follow-up with her orthopedic surgeon in two weeks. She has family support at home, and all necessary pain medications and supplies were arranged prior to discharge. Her final lab results were within normal limits, indicating a low risk of complications."
]


# Prepare inputs for the model
inputs = tokenizer(clinical_notes, truncation=True, max_length=512, padding=True, return_tensors='pt')

# List to store predictions from each fold
all_predictions = []

# Make predictions using each model
for model_path in model_paths:
    # Load the saved model
    model = XLNetForSequenceClassification.from_pretrained(model_path)
    model.eval()  # Set the model to evaluation mode

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predictions = logits.argmax(-1).numpy()  # Get the predicted class labels
        all_predictions.append(predictions)

# Convert list of predictions to a NumPy array
all_predictions = np.array(all_predictions)

# Average the predictions across all folds
# Using np.mean with axis=0 to get the mean prediction for each instance
average_predictions = np.mean(all_predictions, axis=0)

# Convert averaged predictions to class labels
# Classify as 'Readmission' if the mean prediction is >= 0.5, else 'No Readmission'
final_predictions = ["Readmission" if pred >= 0.5 else "No Readmission" for pred in average_predictions]

# Display results
for note, label in zip(clinical_notes, final_predictions):
    print(f"Clinical Note: '{note}' => Predicted Label: '{label}'")


Clinical Note: 'The patient is a 65-year-old female with a history of congestive heart failure and chronic obstructive pulmonary disease (COPD). She was admitted for acute exacerbation of COPD, presenting with severe shortness of breath and low oxygen saturation. During her hospital stay, her condition required frequent adjustments to her oxygen therapy and multiple high-dose steroid treatments. The discharge plan included oxygen therapy at home and follow-up in one week. However, she lives alone and expressed concerns about managing her medications and oxygen setup. Her last lab results showed borderline low sodium and elevated blood urea nitrogen, with fluctuating blood pressure readings. She also had a recent hospitalization three weeks ago for a similar issue.' => Predicted Label: 'Readmission'
Clinical Note: 'The patient is a 45-year-old male with hypertension and Type 2 diabetes, who was admitted for routine monitoring after an elective orthopedic surgery. His hospital course was

In [ ]:
import pandas as pd
import torch
from transformers import XLNetTokenizer, XLNetForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import Dataset
import os
import itertools

# Load the dataset
file_path = '/content/drive/Shared drives/DATA298B/Readmission/Dataset/cleaned_clinical_notes_readmission.csv'
data = pd.read_csv(file_path)

# Initialize the tokenizer
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')

# Define a custom dataset class with contiguous tensors
class ReadmissionDataset(Dataset):
    def __init__(self, encodings, labels):
        # Ensure all tensors in encodings and labels are contiguous
        self.encodings = {key: torch.tensor(val).contiguous() for key, val in encodings.items()}
        self.labels = torch.tensor(labels).contiguous()

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

# Define hyperparameters to tune
hyperparameters = {
    'num_train_epochs': [4],
    'per_device_train_batch_size': [16],
    'learning_rate': [3e-5],
}

# Create all combinations of hyperparameters
param_combinations = list(itertools.product(*hyperparameters.values()))

# Stratified cross-validation setup
k_folds = 5
skf = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=42)
metrics_per_combination = {}

# Folder to save the model
model_save_path = '/content/drive/Shared drives/DATA298B/Readmission/Dataset/saved_model7'
os.makedirs(model_save_path, exist_ok=True)

# Perform cross-validation with hyperparameter tuning
for param_idx, params in enumerate(param_combinations):
    print(f"\nTraining with hyperparameters: {params}")

    num_train_epochs, per_device_train_batch_size, learning_rate = params
    fold_metrics = []

    for fold, (train_index, test_index) in enumerate(skf.split(data, data['readmission'])):
        print(f"\nFold {fold + 1}/{k_folds}")

        # Split the data
        train_texts, test_texts = data['clinical_notes'].iloc[train_index].tolist(), data['clinical_notes'].iloc[test_index].tolist()
        train_labels, test_labels = data['readmission'].iloc[train_index].tolist(), data['readmission'].iloc[test_index].tolist()

        # Tokenize
        train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
        test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=512)

        # Create dataset
        train_dataset = ReadmissionDataset(train_encodings, train_labels)
        test_dataset = ReadmissionDataset(test_encodings, test_labels)

        # Load the model
        model = XLNetForSequenceClassification.from_pretrained('xlnet-base-cased', num_labels=2)

        # Make all model parameters contiguous
        for param in model.parameters():
            param.data = param.data.contiguous()

        # Set up training arguments
        training_args = TrainingArguments(
            output_dir=f'./results_fold_{fold}',
            num_train_epochs=num_train_epochs,
            per_device_train_batch_size=per_device_train_batch_size,
            per_device_eval_batch_size=16,
            learning_rate=learning_rate,
            evaluation_strategy="no",
            report_to="none",
        )

        # Define the Trainer
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=test_dataset,
        )

        # Train the model
        trainer.train()

        # Save the model after training in each fold
        model.save_pretrained(os.path.join(model_save_path, f'model_fold_{fold + 1}_params_{param_idx}'))
        tokenizer.save_pretrained(os.path.join(model_save_path, f'model_fold_{fold + 1}_params_{param_idx}'))

        # Evaluate the model
        predictions = trainer.predict(test_dataset)
        pred_labels = predictions.predictions.argmax(-1)

        # Calculate metrics
        report = classification_report(test_labels, pred_labels, output_dict=True)
        fold_metrics.append(report)

    # Aggregate metrics for the hyperparameter configuration
    avg_metrics = {
        "precision": sum(d["weighted avg"]["precision"] for d in fold_metrics) / k_folds,
        "recall": sum(d["weighted avg"]["recall"] for d in fold_metrics) / k_folds,
        "f1-score": sum(d["weighted avg"]["f1-score"] for d in fold_metrics) / k_folds,
        "accuracy": sum(d["accuracy"] for d in fold_metrics) / k_folds,
    }

    metrics_per_combination[param_idx] = avg_metrics
    print(f"\nAverage Metrics for combination {param_idx}:")
    print(f"Precision: {avg_metrics['precision']:.4f}")
    print(f"Recall: {avg_metrics['recall']:.4f}")
    print(f"F1-score: {avg_metrics['f1-score']:.4f}")
    print(f"Accuracy: {avg_metrics['accuracy']:.4f}")

# Find the best hyperparameter combination
best_combination_idx = max(metrics_per_combination, key=lambda k: metrics_per_combination[k]['f1-score'])
best_metrics = metrics_per_combination[best_combination_idx]
print(f"\nBest hyperparameter combination: {param_combinations[best_combination_idx]}")
print(f"Best F1-score: {best_metrics['f1-score']:.4f}")


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(



Training with hyperparameters: (4, 16, 3e-05)

Fold 1/5


pytorch_model.bin:   0%|          | 0.00/467M [00:00<?, ?B/s]

Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'logits_proj.weight', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss


...

Step,Training Loss



Fold 5/5


Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'logits_proj.weight', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss



Average Metrics for combination 0:
Precision: 0.8292
Recall: 0.8754
F1-score: 0.8418
Accuracy: 0.8754

Best hyperparameter combination: (4, 16, 3e-05)
Best F1-score: 0.8418


In [ ]:
import torch
import numpy as np
from transformers import XLNetTokenizer, XLNetForSequenceClassification

# Specify the number of folds
num_folds = 5  # Adjust this based on the number of folds used during training
model_paths = [f'/content/drive/Shared drives/DATA298B/Readmission/Dataset/saved_model5/model_fold_{i+1}_params_0' for i in range(num_folds)]

# Load the tokenizer
tokenizer = XLNetTokenizer.from_pretrained(model_paths[0])  # Use the tokenizer from any model

# Example clinical notes for prediction
clinical_notes = [

                  "The patient, identified as a 50-year-old male (Patient ID 10059, Hospital Admission ID 122098), was admitted to our facility on 2000-08-22 under the classification of emergency. The patient presented to the hospital via emergency room admit, indicating a possible referral from another healthcare facility or transfer from emergency services. After completing the necessary course of care, the patient was eventually discharged to home. Their hospital stay was covered under the Medicare insurance plan, which facilitated comprehensive treatment. Upon admission, the primary diagnosis was determined to be lower gi bleed (also referred to as Cirrhosis of liver NOS). Given the severity of the patient’s symptoms, immediate medical attention was required. The patient initially spent approximately 3.083333333333333 hours in the emergency department, during which time they were stabilized and subjected to a range of diagnostic tests. Due to the critical nature of their condition, it was decided that further intensive monitoring and intervention were necessary. The patient was transferred to the carevue ICU, where they remained for 1.7806 days. During their ICU stay, the patient was closely monitored and received the appropriate medical interventions tailored to their condition. Notably, the patient was administered sodium chloride 0.9%  flush, categorized as a MAIN medication, to manage their condition effectively. The medication was delivered via IV, ensuring optimal therapeutic benefit. The prescribed regimen lasted for 6 days, with a total administered dosage amounting to 3.0 units. The choice and administration of this medication were crucial in stabilizing the patient's condition and promoting recovery. The patient remained in the hospital for a total of 7 days, during which time their progress was closely monitored. Upon discharge, the patient was deemed stable and capable of continuing recovery at home. This suggests that the initial treatment plan was effective and that the patient adhered well to post-discharge care protocols. Overall, the patient responded positively to the interventions provided during their stay, and their eventual discharge was a result of significant clinical improvement. Ongoing outpatient follow-up will ensure that any further complications are addressed promptly.",
                  "The patient, identified as a 70-year-old male (Patient ID 10094, Hospital Admission ID 122928), was admitted to our facility on 2020-03-15 under the classification of emergency. The patient presented to the hospital via emergency room admit, indicating a possible referral from another healthcare facility or transfer from emergency services. After completing the necessary course of care, the patient was eventually discharged to home. Their hospital stay was covered under the Medicare insurance plan, which facilitated comprehensive treatment. Upon admission, the primary diagnosis was determined to be sepsis;telemetry (also referred to as Septicemia NOS). Given the severity of the patient’s symptoms, immediate medical attention was required. The patient initially spent approximately 7.733333333333333 hours in the emergency department, during which time they were stabilized and subjected to a range of diagnostic tests. Due to the critical nature of their condition, it was decided that further intensive monitoring and intervention were necessary. The patient was transferred to the carevue ICU, where they remained for 4.1014 days. During their ICU stay, the patient was closely monitored and received the appropriate medical interventions tailored to their condition. Notably, the patient was administered prednisone, categorized as a MAIN medication, to manage their condition effectively. The medication was delivered via PO, ensuring optimal therapeutic benefit. The prescribed regimen lasted for 1 days, with a total administered dosage amounting to 5.0 units. The choice and administration of this medication were crucial in stabilizing the patient's condition and promoting recovery. The patient remained in the hospital for a total of 5 days, during which time their progress was closely monitored. Upon discharge, the patient was deemed stable and capable of continuing recovery at home. This suggests that the initial treatment plan was effective and that the patient adhered well to post-discharge care protocols. Overall, the patient responded positively to the interventions provided during their stay, and their eventual discharge was a result of significant clinical improvement. Ongoing outpatient follow-up will ensure that any further complications are addressed promptly."
]

# Prepare inputs for the model
inputs = tokenizer(clinical_notes, truncation=True, max_length=512, padding=True, return_tensors='pt')

# List to store predictions from each fold
all_predictions = []

# Make predictions using each model
for model_path in model_paths:
    # Load the saved model
    model = XLNetForSequenceClassification.from_pretrained(model_path)
    model.eval()  # Set the model to evaluation mode

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predictions = logits.argmax(-1).numpy()  # Get the predicted class labels
        all_predictions.append(predictions)

# Convert list of predictions to a NumPy array
all_predictions = np.array(all_predictions)

# Average the predictions across all folds
# Using np.mean with axis=0 to get the mean prediction for each instance
average_predictions = np.mean(all_predictions, axis=0)

# Convert averaged predictions to class labels
# Classify as 'Readmission' if the mean prediction is >= 0.5, else 'No Readmission'
final_predictions = ["Readmission" if pred >= 0.5 else "No Readmission" for pred in average_predictions]

# Display results
for note, label in zip(clinical_notes, final_predictions):
    print(f"Clinical Note: '{note}' => Predicted Label: '{label}'")


Clinical Note: 'The patient, identified as a 50-year-old male (Patient ID 10059, Hospital Admission ID 122098), was admitted to our facility on 2000-08-22 under the classification of emergency. The patient presented to the hospital via emergency room admit, indicating a possible referral from another healthcare facility or transfer from emergency services. After completing the necessary course of care, the patient was eventually discharged to home. Their hospital stay was covered under the Medicare insurance plan, which facilitated comprehensive treatment. Upon admission, the primary diagnosis was determined to be lower gi bleed (also referred to as Cirrhosis of liver NOS). Given the severity of the patient’s symptoms, immediate medical attention was required. The patient initially spent approximately 3.083333333333333 hours in the emergency department, during which time they were stabilized and subjected to a range of diagnostic tests. Due to the critical nature of their condition, it

In [ ]:
import pandas as pd
import torch
from transformers import XLNetTokenizer, XLNetForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import Dataset
import os
import itertools

# Load the dataset
file_path = '/content/drive/Shared drives/DATA298B/Readmission/Dataset/cleaned_clinical_notes_readmission.csv'
data = pd.read_csv(file_path)

# Initialize the tokenizer
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')

# Define a custom dataset class with contiguous tensors
class ReadmissionDataset(Dataset):
    def __init__(self, encodings, labels):
        # Ensure all tensors in encodings and labels are contiguous
        self.encodings = {key: torch.tensor(val).contiguous() for key, val in encodings.items()}
        self.labels = torch.tensor(labels).contiguous()

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

# Define hyperparameters to tune
hyperparameters = {
    'num_train_epochs': [4],
    'per_device_train_batch_size': [8],
    'learning_rate': [3e-5],
}

# Create all combinations of hyperparameters
param_combinations = list(itertools.product(*hyperparameters.values()))

# Stratified cross-validation setup
k_folds = 5
skf = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=42)
metrics_per_combination = {}

# Folder to save the model
model_save_path = '/content/drive/Shared drives/DATA298B/Readmission/Dataset/saved_model6'
os.makedirs(model_save_path, exist_ok=True)

# Perform cross-validation with hyperparameter tuning
for param_idx, params in enumerate(param_combinations):
    print(f"\nTraining with hyperparameters: {params}")

    num_train_epochs, per_device_train_batch_size, learning_rate = params
    fold_metrics = []

    for fold, (train_index, test_index) in enumerate(skf.split(data, data['readmission'])):
        print(f"\nFold {fold + 1}/{k_folds}")

        # Split the data
        train_texts, test_texts = data['clinical_notes'].iloc[train_index].tolist(), data['clinical_notes'].iloc[test_index].tolist()
        train_labels, test_labels = data['readmission'].iloc[train_index].tolist(), data['readmission'].iloc[test_index].tolist()

        # Tokenize
        train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
        test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=512)

        # Create dataset
        train_dataset = ReadmissionDataset(train_encodings, train_labels)
        test_dataset = ReadmissionDataset(test_encodings, test_labels)

        # Load the model
        model = XLNetForSequenceClassification.from_pretrained('xlnet-base-cased', num_labels=2)

        # Make all model parameters contiguous
        for param in model.parameters():
            param.data = param.data.contiguous()

        # Set up training arguments
        training_args = TrainingArguments(
            output_dir=f'./results_fold_{fold}',
            num_train_epochs=num_train_epochs,
            per_device_train_batch_size=per_device_train_batch_size,
            per_device_eval_batch_size=16,
            learning_rate=learning_rate,
            evaluation_strategy="no",
            report_to="none",
        )

        # Define the Trainer
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=test_dataset,
        )

        # Train the model
        trainer.train()

        # Save the model after training in each fold
        model.save_pretrained(os.path.join(model_save_path, f'model_fold_{fold + 1}_params_{param_idx}'))
        tokenizer.save_pretrained(os.path.join(model_save_path, f'model_fold_{fold + 1}_params_{param_idx}'))

        # Evaluate the model
        predictions = trainer.predict(test_dataset)
        pred_labels = predictions.predictions.argmax(-1)

        # Calculate metrics
        report = classification_report(test_labels, pred_labels, output_dict=True)
        fold_metrics.append(report)

    # Aggregate metrics for the hyperparameter configuration
    avg_metrics = {
        "precision": sum(d["weighted avg"]["precision"] for d in fold_metrics) / k_folds,
        "recall": sum(d["weighted avg"]["recall"] for d in fold_metrics) / k_folds,
        "f1-score": sum(d["weighted avg"]["f1-score"] for d in fold_metrics) / k_folds,
        "accuracy": sum(d["accuracy"] for d in fold_metrics) / k_folds,
    }

    metrics_per_combination[param_idx] = avg_metrics
    print(f"\nAverage Metrics for combination {param_idx}:")
    print(f"Precision: {avg_metrics['precision']:.4f}")
    print(f"Recall: {avg_metrics['recall']:.4f}")
    print(f"F1-score: {avg_metrics['f1-score']:.4f}")
    print(f"Accuracy: {avg_metrics['accuracy']:.4f}")

# Find the best hyperparameter combination
best_combination_idx = max(metrics_per_combination, key=lambda k: metrics_per_combination[k]['f1-score'])
best_metrics = metrics_per_combination[best_combination_idx]
print(f"\nBest hyperparameter combination: {param_combinations[best_combination_idx]}")
print(f"Best F1-score: {best_metrics['f1-score']:.4f}")


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(



Training with hyperparameters: (4, 8, 3e-05)

Fold 1/5


Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'logits_proj.weight', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss



Fold 2/5


...

Step,Training Loss



Fold 5/5


Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'logits_proj.weight', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss



Average Metrics for combination 0:
Precision: 0.9980
Recall: 0.9979
F1-score: 0.9979
Accuracy: 0.9979

Best hyperparameter combination: (4, 8, 3e-05)
Best F1-score: 0.9979


In [ ]:
import torch
import numpy as np
from transformers import XLNetTokenizer, XLNetForSequenceClassification

# Specify the number of folds
num_folds = 5  # Adjust this based on the number of folds used during training
model_paths = [f'/content/drive/Shared drives/DATA298B/Readmission/Dataset/saved_model6/model_fold_{i+1}_params_0' for i in range(num_folds)]

# Load the tokenizer
tokenizer = XLNetTokenizer.from_pretrained(model_paths[0])  # Use the tokenizer from any model

# Example clinical notes for prediction
clinical_notes = [

                  "The patient, identified as a 50-year-old male (Patient ID 10059, Hospital Admission ID 122098), was admitted to our facility on 2000-08-22 under the classification of emergency. The patient presented to the hospital via emergency room admit, indicating a possible referral from another healthcare facility or transfer from emergency services. After completing the necessary course of care, the patient was eventually discharged to home. Their hospital stay was covered under the Medicare insurance plan, which facilitated comprehensive treatment. Upon admission, the primary diagnosis was determined to be lower gi bleed (also referred to as Cirrhosis of liver NOS). Given the severity of the patient’s symptoms, immediate medical attention was required. The patient initially spent approximately 3.083333333333333 hours in the emergency department, during which time they were stabilized and subjected to a range of diagnostic tests. Due to the critical nature of their condition, it was decided that further intensive monitoring and intervention were necessary. The patient was transferred to the carevue ICU, where they remained for 1.7806 days. During their ICU stay, the patient was closely monitored and received the appropriate medical interventions tailored to their condition. Notably, the patient was administered sodium chloride 0.9%  flush, categorized as a MAIN medication, to manage their condition effectively. The medication was delivered via IV, ensuring optimal therapeutic benefit. The prescribed regimen lasted for 6 days, with a total administered dosage amounting to 3.0 units. The choice and administration of this medication were crucial in stabilizing the patient's condition and promoting recovery. The patient remained in the hospital for a total of 7 days, during which time their progress was closely monitored. Upon discharge, the patient was deemed stable and capable of continuing recovery at home. This suggests that the initial treatment plan was effective and that the patient adhered well to post-discharge care protocols. Overall, the patient responded positively to the interventions provided during their stay, and their eventual discharge was a result of significant clinical improvement. Ongoing outpatient follow-up will ensure that any further complications are addressed promptly.",
                  "The patient, identified as a 70-year-old male (Patient ID 10094, Hospital Admission ID 122928), was admitted to our facility on 2020-03-15 under the classification of emergency. The patient presented to the hospital via emergency room admit, indicating a possible referral from another healthcare facility or transfer from emergency services. After completing the necessary course of care, the patient was eventually discharged to home. Their hospital stay was covered under the Medicare insurance plan, which facilitated comprehensive treatment. Upon admission, the primary diagnosis was determined to be sepsis;telemetry (also referred to as Septicemia NOS). Given the severity of the patient’s symptoms, immediate medical attention was required. The patient initially spent approximately 7.733333333333333 hours in the emergency department, during which time they were stabilized and subjected to a range of diagnostic tests. Due to the critical nature of their condition, it was decided that further intensive monitoring and intervention were necessary. The patient was transferred to the carevue ICU, where they remained for 4.1014 days. During their ICU stay, the patient was closely monitored and received the appropriate medical interventions tailored to their condition. Notably, the patient was administered prednisone, categorized as a MAIN medication, to manage their condition effectively. The medication was delivered via PO, ensuring optimal therapeutic benefit. The prescribed regimen lasted for 1 days, with a total administered dosage amounting to 5.0 units. The choice and administration of this medication were crucial in stabilizing the patient's condition and promoting recovery. The patient remained in the hospital for a total of 5 days, during which time their progress was closely monitored. Upon discharge, the patient was deemed stable and capable of continuing recovery at home. This suggests that the initial treatment plan was effective and that the patient adhered well to post-discharge care protocols. Overall, the patient responded positively to the interventions provided during their stay, and their eventual discharge was a result of significant clinical improvement. Ongoing outpatient follow-up will ensure that any further complications are addressed promptly."
]

# Prepare inputs for the model
inputs = tokenizer(clinical_notes, truncation=True, max_length=512, padding=True, return_tensors='pt')

# List to store predictions from each fold
all_predictions = []

# Make predictions using each model
for model_path in model_paths:
    # Load the saved model
    model = XLNetForSequenceClassification.from_pretrained(model_path)
    model.eval()  # Set the model to evaluation mode

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predictions = logits.argmax(-1).numpy()  # Get the predicted class labels
        all_predictions.append(predictions)

# Convert list of predictions to a NumPy array
all_predictions = np.array(all_predictions)

# Average the predictions across all folds
# Using np.mean with axis=0 to get the mean prediction for each instance
average_predictions = np.mean(all_predictions, axis=0)

# Convert averaged predictions to class labels
# Classify as 'Readmission' if the mean prediction is >= 0.5, else 'No Readmission'
final_predictions = ["Readmission" if pred >= 0.5 else "No Readmission" for pred in average_predictions]

# Display results
for note, label in zip(clinical_notes, final_predictions):
    print(f"Clinical Note: '{note}' => Predicted Label: '{label}'")


Clinical Note: 'The patient, identified as a 50-year-old male (Patient ID 10059, Hospital Admission ID 122098), was admitted to our facility on 2000-08-22 under the classification of emergency. The patient presented to the hospital via emergency room admit, indicating a possible referral from another healthcare facility or transfer from emergency services. After completing the necessary course of care, the patient was eventually discharged to home. Their hospital stay was covered under the Medicare insurance plan, which facilitated comprehensive treatment. Upon admission, the primary diagnosis was determined to be lower gi bleed (also referred to as Cirrhosis of liver NOS). Given the severity of the patient’s symptoms, immediate medical attention was required. The patient initially spent approximately 3.083333333333333 hours in the emergency department, during which time they were stabilized and subjected to a range of diagnostic tests. Due to the critical nature of their condition, it